# CIS 545 - Big Data Analytics - Fall 2019

# Homework 2: Querying Linked (LinkedIn) Data

# Due October 11, 2019 at 10pm

Have you ever wondered about (1) what it takes to be a data scientist or "data person", and (2) how social networks and recommender systems work?

This homework is focused on (1) working with hierarchical data stored in dataframes, (2) traversing relationships among data, including graph data, (3) understanding a bit about performance.

We will focus on questions about data scientists from "our" crawl of the LinkedIn dataset, which was also used in the Lecture (module) 2 extended notebook.

In [0]:
!pip install pymongo[tls,srv]
!pip install swifter
!pip install lxml

In [0]:
import pandas as pd
import numpy as np
import json
import sqlite3
from lxml import etree
import urllib
import zipfile

import time
import swifter
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError, OperationFailure

# Step 0: Acquire and load data

We need to pull the zipfile with LinkedIn data from Amazon S3 (where it is shared) to your local machine or the Google Colab cloud-hosted machine.  Only when the data is local can we efficiently parse it (and we'll read directly out of a zip file).

The zip file contains three files with the same schema.  You can start with the `tiny` instance to test your queries, then go on to `small`.  If you're brave and have a lot of time feel free to use the full file.

**We will grade your homework using `small`. Hidden test 0.0 will override your file selection, so as long as you do not change the file selection in a cell that comes after that one, you will be fine.**

* `linkedin.json` (3M records)
* `linkedin_small.json` (100K records)
* `linkedin_tiny.json` (10K records)

The cell below will download a 3GB file to your Google Cloud. It may take a while. You do not need to modify the two cells below.

In [0]:
url = 'https://upenn-bigdataanalytics.s3.amazonaws.com/linkedin.zip'
filehandle, _ = urllib.request.urlretrieve(url,filename='local.zip')
filehandle = 'local.zip'

In [0]:
# What's the zip file actually called locally?
filehandle

![alt text](https://)The cell below creates pointers to the three versions of our dataset. To switch between them, simply change the `file` variable in the cell after the cell below.

In [0]:
def fetch_file(fname):
    zip_file_object = zipfile.ZipFile(filehandle, 'r')
    for file in zip_file_object.namelist():
        file = zip_file_object.open(file)
        if file.name == fname: return file
    return None
    
linkedin_tiny = fetch_file('linkedin_tiny.json')
linkedin_small = fetch_file('linkedin_small.json')
linkedin_huge = fetch_file('linkedin.json')

In [0]:
# CIS 545 Hidden Test 0.0 - please do not modify or delete this cell!

# Set the input file to process
file = linkedin_small

## Step 0.1:  Store data in dataframes

In the cell below, adapt the data loading code from the [in-class notebook](https://colab.research.google.com/drive/1V-QGYHI3YdLVv3sjnfQn7xmy6xEdjJnm).  You will need the function that extracts relations from JSON files and the function that converts relations to dataframes. Read in a maximum of 20000 people. Put the code that reads a line of the file, extracts the relations, removes the interval field, and stores the field information with a try statement, just in case. In the error case, just use a `pass` command to move on. At the end of the next cell, you should have nine dataframes with the following names:

1. `people_df`
2. `names_df`
3. `education_df`
4. `groups_df`
5. `skills_df`
6. `experience_df`
7. `honors_df`
8. `also_view_df`
9. `events_df`

In [0]:
# TODO: Adapt the data loading code from class
START = 0
LIMIT = 20000

def get_df(rel):
    ret = pd.DataFrame(rel).fillna('')
    for k in ret.keys():
        ret[k] = ret[k].astype(str)
    return ret

def extract_relation(rel, name):
    '''
    Pull out a nested list that has a key, and return it as a list
    of dictionaries suitable for treating as a relation / dataframe
    '''
    # We'll return a list
    ret  = []
    if name in rel:
        ret2 = rel.pop(name)
        try:
            # Try to parse the string as a dictionary
            ret2 = json.loads(ret2.replace('\'','\"'))
        except:
            # If we get an error in parsing, we'll leave as a string
            pass
        
        # If it's a dictionary, add it to our return results after
        # adding a key to the parent
        if isinstance(ret2, dict):
            item = ret2
            item['person'] = rel['_id']
            ret.append(item)
        else:
            # If it's a list, iterate over each item
            index = 0
            for r in ret2:
                item = r
                if not isinstance(item, dict):
                    item = {'person': rel['_id'], 'value': item}
                else:
                    item['person'] = rel['_id']
                    
                # A fix to a typo in the data
                if 'affilition' in item:
                    item['affiliation'] = item.pop('affilition')
                    
                item['pos'] = index
                index = index + 1
                ret.append(item)
    return ret

#Parsing through the data
names = []
people = []
groups = []
education = []
skills = []
experience = []
honors = []
also_view = []
events = []

lines = []
i = 1
for line in file:
    if i > START + LIMIT:
        break
    elif i >= START:
        person = json.loads(line)

        # By inspection, all of these are nested dictionary or list content
        nam = extract_relation(person, 'name')
        edu = extract_relation(person, 'education')
        grp = extract_relation(person, 'group')
        skl = extract_relation(person, 'skills')
        exp  = extract_relation(person, 'experience')
        hon = extract_relation(person, 'honors')
        als = extract_relation(person, 'also_view')
        eve = extract_relation(person, 'events')
        
        # This doesn't seem relevant and it's the only
        # non-string field that's sometimes null
        if 'interval' in person:
            person.pop('interval')
        
        lines.append(person)
        names = names + nam
        education = education + edu
        groups  = groups + grp
        skills = skills + skl
        experience = experience + exp
        honors = honors + hon
        also_view = also_view + als
        events = events + eve
        
    i = i + 1

people_df = get_df(pd.DataFrame(lines))
names_df = get_df(pd.DataFrame(names))
education_df = get_df(pd.DataFrame(education))
groups_df = get_df(pd.DataFrame(groups))
skills_df = get_df(pd.DataFrame(skills))
experience_df = get_df(pd.DataFrame(experience))
honors_df = get_df(pd.DataFrame(honors))
also_view_df = get_df(pd.DataFrame(also_view))
events_df = get_df(pd.DataFrame(events))
# YOUR CODE HERE
#raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 0.1 - please do not modify or delete this cell!
display(skills_df)
display(experience_df)

In [0]:
# CIS 545 Hidden Test 0.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 0.1.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 0.1.3 - please do not modify or delete this cell!


## Step 0.2: Convert to SQL

Next save the data to SQLite...  Again, using the same approach as in the sample notebook.

In [0]:
conn = sqlite3.connect('linkedin.db')

people_df.to_sql('people', conn, if_exists='replace', index=False)
names_df.to_sql('names', conn, if_exists='replace', index=False)
education_df.to_sql('education', conn, if_exists='replace', index=False)
groups_df.to_sql('groups', conn, if_exists='replace', index=False)
skills_df.to_sql('skills', conn, if_exists='replace', index=False)
experience_df.to_sql('experience', conn, if_exists='replace', index=False)
honors_df.to_sql('honors', conn, if_exists='replace', index=False)
also_view_df.to_sql('also_view', conn, if_exists='replace', index=False)
events_df.to_sql('events', conn, if_exists='replace', index=False)

# YOUR CODE HERE
#raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 0.2.1 - please do not modify or delete this cell!

people_df.describe()
names_df.describe()

In [0]:
# CIS 545 Sanity Check 0.2.2 - please do not modify or delete this cell!

skills_df.describe()

In [0]:
# CIS 545 Sanity Check 0.2.3 - please do not modify or delete this cell!

experience_df.describe()

# Step 1: What is a data scientist?

In this homework, we will use LinkedIn to analyze what it means to be a data scientist (as of a few years ago).

## Step 1.1: What are common skills for data scientists?

Our first question is:  for anyone who's job revolves around data (database administrators, data curators, data engineers, data scientists), *what are the most common skills*?

### Step 1.1.1: Collect skills (Pandas)

Complete the `collect_skills` function below. This and the other functions in this homework allow us to evaluate the correctness of your queries even if your data do not match ours. The function should:

1. Using `experience_df`, find all people with a position containing "data" in the title. Remember upper versus lower case.
2. Using `skills_df`, find all people with "data science" as a skill. Again, remember to account for case.
3. For all of the unique people found in steps 1 and 2, find the rest of their skills
4. Return a dataframe of the top 15 skills, by frequency  (see pandas.DataFrame.sort_values).  The columns should be called `skill` (the name of the skill) and `scientists` (the count of the number of data scientists with this skill).

In [0]:
# TODO: Find the top 15 skills for data scientists (Pandas)
def collect_skills(experience_df, people_df, skills_df):
    #find all people with data in name:
    experience_df['title'] = experience_df['title'].apply(lambda x: x.lower())
    people_with_data = experience_df[experience_df['title'].str.contains(r'.*(data.*)')]['person']
    display(people_with_data)

    skills_df['value'] = skills_df['value'].apply(lambda x: x.lower())
    people_with_data_science = skills_df[skills_df['value'].str.contains(r'.*(data science.*)')]['person']
    display(people_with_data_science)

    unique_people = pd.concat([people_with_data, people_with_data_science]).to_frame()
    #unique_people = people_with_data.merge(people_with_data_science, how = 'outer', left_on = 'person', right_on = 'person').to_frame()['person'].drop_duplicates()
    #display(unique_people)

    new_skills = unique_people.merge(skills_df, how = 'inner', left_on = 'person', right_on = 'person').rename(columns={'value':'skill'}).drop_duplicates()
    display(new_skills)
    
    #top_skills = new_skills.groupby(['value'], as_index = False).count()
    top_skills = new_skills.groupby('skill').skill.count().nlargest(15).to_frame('scientists').reset_index()
    display(top_skills)

    return top_skills
    # YOUR CODE HERE
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.1.1 - please do not modify or delete this cell!
top_skills_df = collect_skills(experience_df, people_df, skills_df)
display(top_skills_df)

if "skill" not in top_skills_df:
    raise AssertionError("skill column not defined")
if "scientists" not in top_skills_df:
    raise AssertionError("scientists column not defined")
if len(top_skills_df) != 15:
    raise AssertionError("dataframe does not have top 15")

In [0]:
# CIS 545 Hidden Test 1.1.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.1.1.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.1.1.3 - please do not modify or delete this cell!


### Step 1.1.2: Top skills (SQL)

Compute the same table as in 1.1.1 using SQL. Store it as `top_skills_sql` but otherwise matching the schema and other properties. Be sure to also save the data to SQLLite in a table called `top_skills`, as we will be testing to see if this table exists.

In [0]:
# TODO: Find the top 15 skills for data scientists (SQL)

# YOUR CODE HERE
conn.execute('''DROP VIEW IF EXISTS experience_sql''')
conn.execute('''CREATE VIEW experience_sql AS
             SELECT DISTINCT person
             FROM experience 
             WHERE LOWER(title) like '%data%'
             ''')

conn.execute('''DROP VIEW IF EXISTS new_skills_sql''')
conn.execute('''CREATE VIEW new_skills_sql AS
             SELECT DISTINCT person
             FROM skills 
             WHERE LOWER(value) LIKE '%data science%' 
             ''')
#unique_people = people_with_data.merge(people_with_data_science, how = 'outer', left_on = 'person', right_on = 'person')['person'].to_frame().drop_duplicates()

conn.execute('''DROP VIEW IF EXISTS newer_skills_sql''')
conn.execute('''CREATE VIEW newer_skills_sql AS
             SELECT DISTINCT e.person
             FROM experience_sql e
             LEFT JOIN new_skills_sql s USING(person)
             UNION ALL
             SELECT DISTINCT e.person
             FROM new_skills_sql s
             LEFT JOIN experience_sql e USING(person)
             ''')

#new_skills = unique_people.merge(skills_df, how = 'left', left_on = 'person', right_on = 'person').dropna().rename(columns={'value':'skills'})

#top_skills = new_skills.groupby('skills').skills.count().nlargest(15).to_frame('scientists').reset_index().rename(columns={"skills": "skill"})
conn.execute('''DROP VIEW IF EXISTS top_skills''')
conn.execute('''CREATE VIEW top_skills AS
             SELECT value as skill, COUNT(value) as scientists
             FROM newer_skills_sql
             INNER JOIN skills ON skills.person = newer_skills_sql.person
             GROUP BY value
             ORDER BY COUNT(value) DESC
             LIMIT 15
             ''')

top_skills_sql = pd.read_sql_query('''
                              SELECT * FROM top_skills
                              ''', conn)

display(top_skills)
#display(unique_df)
#display(skills_df)
#display(experience_df)
#raise NotImplementedError()
#display(top_skills_sql)

In [0]:
#CIS 545 Sanity Check 1.1.2 - please do not modify or delete this cell!



if "skill" not in top_skills_sql:
    raise AssertionError("skill column not defined")
if "scientists" not in top_skills_sql:
    raise AssertionError("scientists column not defined")
if len(top_skills_df) < 1:
    raise AssertionError("dataframe has no results")  
if len(top_skills_sql.merge(top_skills_df)) != len(top_skills_sql):
    raise AssertionError("Pandas and SQL versions are not of the same length")

In [0]:
# CIS 545 Hidden Test 1.1.2 - please do not modify or delete this cell!


## Step 1.2: What are common titles for those with data science skills?

Complete the `collect_titles` function below that aggregates the most recent titles of people with data science skills. This function should use the given dataframes as input and return a two column dataframe: one column called `title` and the other called `count`. You should only consider people who have at least `min_skills` of the top skills for a data scientist. You should also only keep those titles that appear at least `min_count` times.

For extra practice, you can also do this in SQL, although we are not grading that.

In [0]:
# TODO: Find the common titles (Pandas)
def collect_titles(top_skills_df, skills_df, people_df, experience_df, min_skills, min_count):
    # YOUR CODE HERE

    #.skills.count().nlargest(15).to_frame('scientists').reset_index().rename(columns={"skills": "skill"})
    #find all the people_ids associated with the top skills
    new_skills = skills_df.merge(top_skills_df, how = 'inner', left_on = 'value', right_on = 'skill')
    new_skills = new_skills.drop(columns=['scientists', 'skill','pos'])
    display(new_skills)

    #filter these people to see that they have 6+ skills
    people_with_min_skills = new_skills.groupby(['person']).count().sort_values(by = 'value').reset_index()
    #display(people_with_min_skills)
    people_with_min_skills = people_with_min_skills[people_with_min_skills['value'] >= min_skills]
    display(people_with_min_skills)

    #append to final dataframe if the title appears more min_count times (2x)
    almost_final_dataframe = experience_df.merge(people_with_min_skills, how = 'inner', left_on = 'person', right_on = 'person')
    display(almost_final_dataframe)
    almost_final_dataframe = almost_final_dataframe[(almost_final_dataframe['end'] == 'Present') | (almost_final_dataframe['pos'] == 0)]
    display(almost_final_dataframe)

    final_dataframe = almost_final_dataframe.groupby(['title']).title.count().to_frame('count').sort_values(by = 'count', ascending = False).reset_index()
    display(final_dataframe)
    final_dataframe = final_dataframe[final_dataframe['count'] >= min_count]
    display(final_dataframe)

    return final_dataframe
    #raise NotImplementedError()

In [0]:

# CIS 545 Sanity Check 1.2 - please do not modify or delete this cell!

ds_titles_df = collect_titles(top_skills_df, skills_df, people_df, experience_df, 6, 2)
display(ds_titles_df)

'''if "title" not in ds_titles_df:
    raise AssertionError("title column not defined")
if "count" not in ds_titles_df:
    raise AssertionError("count column not defined")
if len(ds_titles_df) < 1:
    raise AssertionError("dataframe has no results")'''

In [0]:
# CIS 545 Hidden Test 1.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.2.3 - please do not modify or delete this cell!


## Step 1.3: Who employs "data people" based on title?

Now let's find the list of companies that have employed people with the above titles, ranked by number of employees who have had these titles.

### Step 1.3.1: Data employers

Complete the `collect_employers` function below that aggregates the employers with positions corresponding to the most recent titles of people with data science skills. This function should use the given dataframes as input and return a two column dataframe: one column called `org` and the other called `people`. Show the names of companies (in field `org`) with at least `min_count` employees who are "data people" (include that count in the `people` column). Order the dataframe by the count of data people in the company in descending order.

In [0]:
# TODO: Find the data employers

def collect_employers(experience_df, ds_titles_df, min_count):
    # YOUR CODE HERE
    new_dataframe = experience_df.merge(ds_titles_df, how = 'inner', left_on = 'title', right_on = 'title').drop(columns = {'desc', 'end', 'person', 'pos', 'start', 'title'})
    display(new_dataframe)

    grouped_dataset = new_dataframe.groupby(['org']).org.count().to_frame('count').sort_values(by='count', ascending=False).reset_index()
    grouped_dataset = grouped_dataset[grouped_dataset['count'] >= min_count].rename(columns = {'count':'people'})
    display(grouped_dataset)
    
    return grouped_dataset

    #.title.count().to_frame('count').sort_values(by = 'count', ascending = False).reset_index()

    #new_dataframe = new_dataframe[new_dataframe['title'] >= min_count]
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.3.1 - please do not modify or delete this cell!

employers_df = collect_employers(experience_df, ds_titles_df, 5)
display(employers_df)

if "IBM" not in employers_df['org'].tolist():
    raise AssertionError("Missing IBM")
    
if employers_df['people'].min() < 4:
    raise AssertionError("Not filtering properly")

In [0]:
# CIS 545 Hidden Test 1.3.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.1.2 - please do not modify or delete this cell!


### Step 1.3.2:  Their employees

Complete the `collect_employees` function below that aggregates the employees of employers with positions corresponding to the most recent titles of people with data science skills. In other words, who are the employees of the data employers you found before and what are their titles? This function should use the given dataframes as input and return the `org`, `family_name`, `given_name`, and `title` of each person.

In [0]:
# TODO: Find the employees of the data employers
def collect_employees(people_df, experience_df, employers_df, names_df,ds_titles_df):

  #find all titles within employers that has ds_titles_df
  new_dataframe = experience_df.merge(ds_titles_df, how = 'inner', left_on = 'title', right_on = 'title')
  #display(new_dataframe)

  #finding all matching employers from new datafram
  newest_datframe = new_dataframe.merge(employers_df, how = 'inner', left_on = 'org', right_on = 'org').drop(columns ={'desc', 'end', 'pos', 'count', 'people'})
  display(newest_datframe)

  final_dataframe = newest_datframe.merge(names_df, how = 'inner', left_on = 'person', right_on = 'person')[['org','family_name','given_name','title']]
  display(final_dataframe)
# YOUR CODE HERE
#raise NotImplementedError()

In [0]:

# CIS 545 Sanity Check 1.3.2 - please do not modify or delete this cell!

title_people_df = collect_employees(people_df, experience_df, employers_df, names_df, ds_titles_df)
display(title_people_df)

'''if len(title_people_df.columns) != 4:
    raise AssertionError('Wrong number of columns. Check schema again')'''

In [0]:
# CIS 545 Hidden Test 1.3.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.3.2.3 - please do not modify or delete this cell!


## Step 1.4: Find peers

In many common social graph settings, we can make recommendations to people based on their similarity with other people. In this case, we define similarity in terms of the number of identical skills.

Suppose A and B have similar skills: A -> X1 and B -> X1, A -> X2 and B -> X2, etc. up to A -> Xk and B -> Xk.

Then given that A and B have similar skills, we might recommend A's employer to B, and vice versa.

### Step 1.4.0: Making the problem tractable in Pandas

Let's consider only the first 100 people in `people_df`.

Find, out of this set, the pairs of people with the most shared/common skills, and return the closest 20 pairs in descending order.  We'll then use this to make a *recommendation* for a potential employer and position to each person.

### Step 1.4.1: Compute the top pairs of peers

Complete the `collect_peers` function below that finds the top `num` pairs of peers. In other words, compare each person with each *other* person, counting the total set of skills in common. This function should use the given dataframes and `num` as input and return a three column dataframe: `person_1`, `person_2`, and `common_skills`. The first two columns should be person IDs and the last column should be the number of skills that this pair of people shares.

Hint: Doing this requires a *Cartesian product*, i.e., every ID paired with every other ID.  Think about how to create a dataframe just with people IDs, then add a field to this dataframe that will let us combine every record with every record.

In [0]:
# TODO: Finish the collect_peers function

people_df_subset = people_df.head(100)

def collect_peers(people_df_subset, skills_df, num):
    people_with_skills = people_df_subset.merge(skills_df, how='inner', left_on = '_id', right_on = 'person')

    #display(people_with_skills)
    subset_of_skills = people_with_skills.merge(people_with_skills, how = 'inner', left_on = 'value', right_on = 'value').rename(columns={ '_id_x':'person_1', '_id_y':'person_2'})
    subset_of_skills = subset_of_skills[subset_of_skills['person_1'] != subset_of_skills['person_2']]
    display(subset_of_skills)

    final_dataframe = subset_of_skills[['person_1', 'person_2', 'value']].groupby(['person_1', 'person_2'])\
                      .value.count().nlargest(20).to_frame('count').sort_values(by='count', ascending=False).rename(columns={ 'count':'common_skills'}).reset_index()

    return final_dataframe
    # YOUR CODE HERE
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.4.1 - please do not modify or delete this cell!

recs_df = collect_peers(people_df_subset, skills_df, 20)
display(recs_df)

if "person_1" not in recs_df:
    raise AssertionError("person_1 column not defined")
if "person_2" not in recs_df:
    raise AssertionError("person_2 column not defined")
if "common_skills" not in recs_df:
    raise AssertionError("common_skills column not defined")
if(len(recs_df) != 20):
    raise AssertionError('Wrong number of rows in recs_df')

In [0]:
# CIS 545 Hidden Test 1.4.1.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.1.2 - please do not modify or delete this cell!


### Step 1.4.2: Get the last jobs

Complete the `last_job` function below that takes `experience_df` as input and returns the `person`, `title`, and `org` corresponding to each person's **last** (most recent) employment experience (three column dataframe).

In [0]:
# TODO: Complete the last_job function

def last_job(experience_df):
    # YOUR CODE HERE
    final_dataframe = experience_df[['person', 'title','pos','org']]
    #display(final_dataframe)
    final_dataframe = final_dataframe[final_dataframe['pos'] == '0']
    #display(final_dataframe)
    final_dataframe = final_dataframe[['person', 'title', 'org']]
    return final_dataframe
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.4.2 - please do not modify or delete this cell!

last_job_df = last_job(experience_df)
display(last_job_df)

if(len(last_job_df.columns) != 3):
    raise AssertionError('Wrong number of columns in last_job_df')

In [0]:
# CIS 545 Hidden Test 1.4.2.1 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.2.2 - please do not modify or delete this cell!


In [0]:
# CIS 545 Hidden Test 1.4.2.3 - please do not modify or delete this cell!


### Step 1.4.3: Recommend jobs

Complete the `recommend_jobs` function below that takes `recs_df`, `names_df`, and `last_job_df` as input and returns for each `person_1`, `person_2`'s most recent `title` and `org`.

In [0]:
# TODO: Complete the recommend_jobs function

def recommend_jobs(recs_df, names_df, last_job_df):
    # YOUR CODE HERE
    final_person1_dataframe = recs_df.merge(last_job_df, how = 'inner', left_on = 'person_1', right_on = 'person')
    #display(final_person1_dataframe)

    final_person_2_dataframe = recs_df.merge(last_job_df, how = 'inner', left_on = 'person_2', right_on = 'person')
    #display(final_person_2_dataframe)
    
    final_dataframe = pd.concat([final_person1_dataframe, final_person_2_dataframe], ignore_index=True)
    display(final_dataframe)

    final_dataframe = final_dataframe.merge(names_df, how = 'inner', left_on = 'person', right_on = 'person')[['family_name', 'given_name', 'org', 'title', 'person_1','person_2', 'common_skills']]
    #final_dataframe = final_dataframe.rename(columns={'_id_x':'person_1', '_id_y':'person_2'})
    final_dataframe = final_dataframe[final_dataframe['person_1'] != final_dataframe['person_2']]
    
    #display(final_dataframe)
    return final_dataframe
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 1.4.3 - please do not modify or delete this cell!

recommended_df = recommend_jobs(recs_df, names_df, last_job_df)
display(recommended_df)

if "family_name" not in recommended_df:
    raise AssertionError("person_1 column not defined")
if "given_name" not in recommended_df:
    raise AssertionError("person_2 column not defined")
if "person_2" not in recommended_df:
    raise AssertionError("common_skills column not defined")
if "org" not in recommended_df:
    raise AssertionError("common_skills column not defined")
if "title" not in recommended_df:
    raise AssertionError("common_skills column not defined")

In [0]:
# CIS 545 Hidden Test 1.4.3 - please do not modify or delete this cell!


# Step 2: Compare Evaluation Orders

This last section relates to our discussions in lecture about computation efficiency with big data.

## Step 2.0: Load custom functions

Let's look at some computation and optimization tasks.  We'll start with the code from our lecture notebooks, which does joins between dataframes.

In [0]:
# Join using nested loops
def merge(S,T,l_on,r_on):
    ret = pd.DataFrame()
    count = 0
    S_ = S.reset_index().drop(columns=['index'])
    T_ = T.reset_index().drop(columns=['index'])
    for s_index in range(0, len(S)):
        for t_index in range(0, len(T)):
            count = count + 1
            if S_.loc[s_index, l_on] == T_.loc[t_index, r_on]:
                ret = ret.append(S_.loc[s_index].append(T_.loc[t_index].drop(labels=r_on)), ignore_index=True)

    print('Merge compared %d tuples'%count)
    return ret
  
# Join using a *map*, which is a kind of in-memory index
# from keys to (single) values
def merge_map(S,T,l_on,r_on):
    ret = pd.DataFrame()
    T_map = {}
    count = 0
    # Take each value in the r_on field, and
    # make a map entry for it
    T_ = T.reset_index().drop(columns=['index'])
    for t_index in range(0, len(T)):
        # Make sure we aren't overwriting an entry!
        assert (T_.loc[t_index,r_on] not in T_map)
        T_map[T_.loc[t_index,r_on]] = T_.loc[t_index]
        count = count + 1

    # Now find matches
    S_ = S.reset_index().drop(columns=['index'])
    for s_index in range(0, len(S)):
        count = count + 1
        if S_.loc[s_index, l_on] in T_map:
                ret = ret.append(S_.loc[s_index].append(T_map[S_.loc[s_index, l_on]].drop(labels=r_on)), ignore_index=True)

    print('Merge compared %d tuples'%count)
    return ret

## Step 2.1: Find an optimal order of evaluation.

Reimplement `recommend_jobs` using the above `merge` or `merge_map` functions instead of Pandas' merge. Try to find the **most efficient** way.  You should start with the dataframes `recs_df`, `names_df`, and `last_job_df` from above. Store your results in `recs_new_df`

In [0]:
# TODO: Reimplement recommend jobs using our custom merge and merge_map functions

def recommend_jobs_new(recs_df, names_df, last_job_df):
    #display(recs_df)
    #display(last_job_df)
    #display(names_df)
    #utilizing merge
    '''
    final_person1_dataframe = merge(recs_df, last_job_df, 'person_1', 'person')
    final_person_2_dataframe = merge(recs_df, last_job_df, 'person_2', 'person')  
    recs_new_df = pd.concat([final_person1_dataframe, final_person_2_dataframe], ignore_index=True)
    recs_new_df = merge(recs_new_df, names_df, 'person_1', 'person')
    #display(recs_new_df)
    recs_new_df = recs_new_df[recs_new_df['person_1'] != recs_new_df['person_2']]
    recs_new_df = recs_new_df.drop(columns=['common_skills', 'person_1'])
    #display(recs_new_df)'''


    #utilizing merge_map
    final_person1_dataframe = merge_map(recs_df, last_job_df, 'person_1', 'person')
    #display(final_person1_dataframe)

    final_person_2_dataframe = merge_map(recs_df, last_job_df, 'person_2', 'person')  
    #display(final_person_2_dataframe)
    
    recs_new_df = pd.concat([final_person1_dataframe, final_person_2_dataframe], ignore_index=True)
    #display(recs_new_df)

    recs_new_df = merge_map(recs_new_df, names_df, 'person_1', 'person')
    display(recs_new_df)
    #recs_new_df_2 = merge_map(recs_new_df, names_df, 'person_2', 'person')
    #recs_new_df = pd.concat([final_person1_dataframe, final_person_2_dataframe], ignore_index=True)
    #recs_new_df = recs_new_df.rename(columns={'_id_x':'person_1', '_id_y':'person_2'})
    recs_new_df = recs_new_df[recs_new_df['person_1'] != recs_new_df['person_2']]
    recs_new_df = recs_new_df.drop(columns=['common_skills', 'person_1'])
    display(recs_new_df)
    
    return recs_new_df
    # YOUR CODE HERE
    #raise NotImplementedError()

In [0]:
# CIS 545 Sanity Check 2.1 - please do not modify or delete this cell!

%%time

recs_new_df = recommend_jobs_new(recs_df, names_df, last_job_df)

if(len(recs_new_df.columns) != 5):
    raise AssertionError('Wrong number of columns in recs_new_df')

# Step 3: Submitting Your Homework

1. When you are done, select “Edit” at the top of the window, **under the filename, not the one that may appear above it**. Then, select “Clear all outputs”. Please do this just before turning is your homework because it reduces the size of your file.


2. In the same menu **under the filename**, select “File” and then “Download .ipynb”. It is very important that you do not change the file name of this downloaded notebook. Make sure that something like “(1)” did not get added to the filename and also that you did not download the .py version. Our autograder can only handle .ipynb files with the correct file name.

3. Compress the ipynb file into a Zip file **hw2.zip**.

4. Go to the [submission site](http://submit.dataanalytics.education), and click on the Google icon.  Log in using your Google@SEAS (if at all possible!) or (if you aren’t an Engineering student) GMail account.  

5. Click on the **Courses** icon at the top, then select **CIS 545** and **Save**. Select **cis545-2019c-hw2** and upload **hw2.zip**.

6. You should see a message on the submission site notifying you about whether your submission passed validation.  You may resubmit as necessary, but may have to withdraw your previous submission in OpenSubmit in order to do so.

**If you have not already, please go to Settings and set your Student ID to your PennID (all numbers)**.